# NSF Grant Program Element Analysis: 2025 vs 2021-2024 Comparison

This notebook analyzes NSF grant program elements to compare funding patterns between 2025 and the 2021-2024 baseline period.

## Analysis Objectives

For each program element name (`pgm_ele_name`), we will compute:
1. The proportion of total funding in 2025 allocated to grants with this program element
2. The percentage of total grants in 2025 which contain this program element
3. The proportion of total funding from 2021-2024 (average) allocated to grants with this program element
4. The percentage of total grants from 2021-2024 (average) which contain this program element

## Step 1: Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
import json
import glob
from tqdm import tqdm

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
def load_nsf_grants_from_year(year):
    """
    Load all NSF grant JSON files for a given year and extract relevant data.
    Returns a DataFrame with awd_amount and pgm_ele_names.
    """
    print(f"Loading NSF grants for {year}...")
    
    # Find all JSON files for this year
    json_pattern = f"./data/nsf/{year}/*.json"
    json_files = glob.glob(json_pattern)
    
    print(f"  - Found {len(json_files)} JSON files")
    
    grants_data = []
    
    for json_file in tqdm(json_files, desc=f"Processing {year}"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                grant_data = json.load(f)
            
            # Extract award amount
            awd_amount = grant_data.get('awd_amount', 0)
            if awd_amount is None:
                awd_amount = 0
            
            # Extract program element names
            pgm_ele_names = []
            pgm_ele = grant_data.get('pgm_ele', [])
            
            if pgm_ele:
                for element in pgm_ele:
                    if isinstance(element, dict) and 'pgm_ele_name' in element:
                        pgm_ele_name = element['pgm_ele_name']
                        if pgm_ele_name and pgm_ele_name.strip():
                            pgm_ele_names.append(pgm_ele_name.strip())
            
            grants_data.append({
                'awd_amount': float(awd_amount),
                'pgm_ele_names': pgm_ele_names,
                'file_path': json_file
            })
            
        except Exception as e:
            print(f"    Error processing {json_file}: {e}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(grants_data)
    
    print(f"  - Successfully processed {len(df)} grants")
    print(f"  - Total funding: ${df['awd_amount'].sum():,.0f}")
    
    return df

In [3]:

# Load data for all years
years = [2021, 2022, 2023, 2024, 2025]
dataframes = {}

for year in years:
    dataframes[year] = load_nsf_grants_from_year(year)
    print()

print("All NSF grant data loaded successfully!")

Loading NSF grants for 2021...
  - Found 12166 JSON files


Processing 2021: 100%|██████████| 12166/12166 [00:02<00:00, 4979.61it/s]


  - Successfully processed 12166 grants
  - Total funding: $8,193,757,121

Loading NSF grants for 2022...
  - Found 11912 JSON files


Processing 2022: 100%|██████████| 11912/11912 [00:02<00:00, 4773.24it/s]


  - Successfully processed 11912 grants
  - Total funding: $7,269,309,476

Loading NSF grants for 2023...
  - Found 12022 JSON files


...

  - Successfully processed 12022 grants
  - Total funding: $7,348,114,730

Loading NSF grants for 2024...
  - Found 11687 JSON files


Processing 2024: 100%|██████████| 11687/11687 [00:02<00:00, 5403.68it/s]


  - Successfully processed 11687 grants
  - Total funding: $6,385,263,306

Loading NSF grants for 2025...
  - Found 9249 JSON files


Processing 2025: 100%|██████████| 9249/9249 [00:01<00:00, 5438.12it/s]


  - Successfully processed 9249 grants
  - Total funding: $4,772,095,689

All NSF grant data loaded successfully!


In [5]:
dataframes.get(2025)[:10]

,awd_amount,pgm_ele_names,file_path
0,1266115.0,"[Evolutionary Processes, Animal Behavior]",./data/nsf/2025/2525976.json
1,999824.0,[S-STEM-Schlr Sci Tech Eng&Math],./data/nsf/2025/2424506.json
2,550000.0,[SOLID STATE & MATERIALS CHEMIS],./data/nsf/2025/2521466.json
3,1132274.0,[Cellular Dynamics and Function],./data/nsf/2025/2517807.json
4,41482.0,[OAC-Advanced Cyberinfrast Core],./data/nsf/2025/2540175.json
5,50000.0,[I-Corps],./data/nsf/2025/2445015.json
6,50000.0,[I-Corps],./data/nsf/2025/2535075.json
7,494999.0,"[CCSS-Comms Circuits & Sens Sys, EPSCoR Co-Funding]",./data/nsf/2025/2432082.json
8,4500000.0,[NSF Research Traineeship (NRT)],./data/nsf/2025/2440647.json
9,44844.0,"[SPSE-Study of Physics of Earth, ANT Earth Sciences]",./data/nsf/2025/2514682.json


## Step 2: Data Preprocessing

In [6]:
# Examine the structure of one dataframe
print("Sample dataframe structure (2021):")
print(f"Columns: {list(dataframes[2021].columns)}")
print(f"\nFirst few rows:")
print(dataframes[2021].head())
print(f"\nData types:")
print(dataframes[2021].dtypes)
print(f"\nSample program element names:")
print(dataframes[2021]['pgm_ele_names'].iloc[0])

Sample dataframe structure (2021):
Columns: ['awd_amount', 'pgm_ele_names', 'file_path']

First few rows:
   awd_amount  \
0    275844.0   
1   9286270.0   
2    146000.0   
3    399095.0   
4    385968.0   

                                                                                pgm_ele_names  \
0                                                [SBIR Phase I, SBIR Outreach & Tech. Assist]   
1                                                            [SSA-Special Studies & Analysis]   
2  [(SPRF-FR) SBE Postdoctoral Res, GVF - Global Venture Fund, SPRF-Broadening Participation]   
3                                        [CFS-Combustion & Fire Systems, Special Initiatives]   
4                                 [Symbiosis Infection & Immunity, Unallocated Program Costs]   

                      file_path  
0  ./data/nsf/2021/2035129.json  
1  ./data/nsf/2021/2048419.json  
2  ./data/nsf/2021/2105307.json  
3  ./data/nsf/2021/2126467.json  
4  ./data/nsf/2021/2121658.json  

Dat

In [7]:
def preprocess_nsf_dataframe(df, year):
    """
    Clean and preprocess the NSF dataframe:
    1. Remove grants with zero award amount
    2. Remove grants with no program elements
    3. Validate data types
    """
    print(f"Preprocessing {year} NSF data...")
    
    # Make a copy to avoid modifying original
    df_clean = df.copy()
    
    initial_rows = len(df_clean)
    
    # Remove grants with zero or null award amount
    df_clean = df_clean[df_clean['awd_amount'].notna()]
    df_clean = df_clean[df_clean['awd_amount'] > 0]
    
    # Remove grants with no program elements
    df_clean = df_clean[df_clean['pgm_ele_names'].apply(lambda x: len(x) > 0)]
    
    final_rows = len(df_clean)
    rows_removed = initial_rows - final_rows
    
    print(f"  - Initial grants: {initial_rows}")
    print(f"  - Final grants: {final_rows}")
    print(f"  - Grants removed: {rows_removed}")
    print(f"  - Total funding: ${df_clean['awd_amount'].sum():,.0f}")
    
    return df_clean

In [8]:
# Preprocess all dataframes
clean_dataframes = {}
for year, df in dataframes.items():
    clean_dataframes[year] = preprocess_nsf_dataframe(df, year)
    print()

print("NSF data preprocessing completed!")

Preprocessing 2021 NSF data...
  - Initial grants: 12166
  - Final grants: 12072
  - Grants removed: 94
  - Total funding: $8,150,654,654

Preprocessing 2022 NSF data...
  - Initial grants: 11912
  - Final grants: 11818
  - Grants removed: 94
  - Total funding: $7,242,058,435

Preprocessing 2023 NSF data...
  - Initial grants: 12022
  - Final grants: 11980
  - Grants removed: 42
  - Total funding: $7,335,036,929

Preprocessing 2024 NSF data...
  - Initial grants: 11687
  - Final grants: 11641
  - Grants removed: 46
  - Total funding: $6,375,623,081

Preprocessing 2025 NSF data...
  - Initial grants: 9249
  - Final grants: 9233
  - Grants removed: 16
  - Total funding: $4,770,031,044

NSF data preprocessing completed!


## Step 3: Program Element Extraction and Normalization

In [9]:
def extract_program_elements_from_dataframe(df, year):
    """
    Extract all unique program element names from a dataframe.
    Program elements are already in list form in pgm_ele_names column.
    """
    print(f"Extracting program elements from {year} data...")
    
    all_program_elements = set()
    
    for pgm_ele_names in df['pgm_ele_names']:
        if pgm_ele_names and len(pgm_ele_names) > 0:
            # Normalize program element names
            normalized_elements = [elem.strip() for elem in pgm_ele_names if elem and elem.strip()]
            all_program_elements.update(normalized_elements)
    
    print(f"  - Found {len(all_program_elements)} unique program elements")
    return all_program_elements

In [10]:
# Extract program elements from each year
yearly_program_elements = {}
for year, df in clean_dataframes.items():
    yearly_program_elements[year] = extract_program_elements_from_dataframe(df, year)

Extracting program elements from 2021 data...
  - Found 470 unique program elements
Extracting program elements from 2022 data...
  - Found 472 unique program elements
Extracting program elements from 2023 data...
  - Found 478 unique program elements
Extracting program elements from 2024 data...
  - Found 472 unique program elements
Extracting program elements from 2025 data...
  - Found 446 unique program elements


In [11]:
# Create master list of all unique program elements
all_unique_program_elements = set()
for elements in yearly_program_elements.values():
    all_unique_program_elements.update(elements)

In [12]:
print(f"\nTotal unique program elements across all years: {len(all_unique_program_elements)}")


Total unique program elements across all years: 639


In [13]:
# Display some sample program elements
sample_elements = list(all_unique_program_elements)[:10]
print("\nSample program elements:")
for elem in sample_elements:
    print(f"  - {elem}")


Sample program elements:
  - Sci of Lrng & Augmented Intel
  - CIS-Civil Infrastructure Syst
  - Cross-Agency Priority (CAP) Go
  - Dimensions of Biodiversity
  - DAT-Democracy AffrmngTchnolgie
  - IceCube Research Support
  - Innovative HPC
  - Methodology, Measuremt & Stats
  - FM-Future Manufacturing
  - ECR-EDU Core Research


## Step 4: Calculate Annual Program Element Metrics

In [14]:
def calculate_program_element_metrics_for_year(df, year, all_program_elements):
    """
    For each program element, calculate:
    - Number of grants containing the program element
    - Total funding for grants containing the program element
    """
    print(f"Calculating metrics for {year}...")
    
    element_metrics = {}
    total_grants = len(df)
    total_funding = df['awd_amount'].sum()
    
    print(f"  - Processing {len(all_program_elements)} program elements for {total_grants} grants")
    
    # Use tqdm for progress bar on large element sets
    for element in tqdm(all_program_elements, desc=f"Processing {year}"):
        # Find grants that contain this program element
        mask = df['pgm_ele_names'].apply(lambda elem_list: element in elem_list)
        grants_with_element = df[mask]
        
        grant_count = len(grants_with_element)
        funding_total = grants_with_element['awd_amount'].sum() if grant_count > 0 else 0
        
        element_metrics[element] = {
            'grant_count': grant_count,
            'funding_total': funding_total,
            'grant_percentage': grant_count / total_grants,
            'funding_proportion': funding_total / total_funding
        }
    
    return element_metrics, total_grants, total_funding

In [15]:

# Calculate metrics for each year
annual_metrics = {}
annual_totals = {}

for year, df in clean_dataframes.items():
    metrics, total_grants, total_funding = calculate_program_element_metrics_for_year(
        df, year, all_unique_program_elements
    )
    annual_metrics[year] = metrics
    annual_totals[year] = {
        'total_grants': total_grants,
        'total_funding': total_funding
    }
    print(f"  - Completed {year}: {total_grants} grants, ${total_funding:,.0f} total funding\n")

print("Annual metrics calculation completed!")

Calculating metrics for 2021...
  - Processing 639 program elements for 12072 grants


Processing 2021: 100%|██████████| 639/639 [00:00<00:00, 887.33it/s]


  - Completed 2021: 12072 grants, $8,150,654,654 total funding

Calculating metrics for 2022...
  - Processing 639 program elements for 11818 grants


Processing 2022: 100%|██████████| 639/639 [00:00<00:00, 968.75it/s]


  - Completed 2022: 11818 grants, $7,242,058,435 total funding

Calculating metrics for 2023...
  - Processing 639 program elements for 11980 grants


...

  - Completed 2023: 11980 grants, $7,335,036,929 total funding

Calculating metrics for 2024...
  - Processing 639 program elements for 11641 grants


Processing 2024: 100%|██████████| 639/639 [00:00<00:00, 998.43it/s] 


  - Completed 2024: 11641 grants, $6,375,623,081 total funding

Calculating metrics for 2025...
  - Processing 639 program elements for 9233 grants


Processing 2025: 100%|██████████| 639/639 [00:00<00:00, 1117.97it/s]

  - Completed 2025: 9233 grants, $4,770,031,044 total funding

Annual metrics calculation completed!


## Step 5: Compute Proportional Comparisons

In [16]:
# Calculate 2021-2024 average metrics
print("Calculating 2021-2024 average metrics...")

baseline_years = [2021, 2022, 2023, 2024]
average_metrics = {}

print(f"Average period (2021-2024):")
for year in baseline_years:
    print(f"  - {year}: {annual_totals[year]['total_grants']:,} grants, ${annual_totals[year]['total_funding']:,.0f} funding")

Calculating 2021-2024 average metrics...
Average period (2021-2024):
  - 2021: 12,072 grants, $8,150,654,654 funding
  - 2022: 11,818 grants, $7,242,058,435 funding
  - 2023: 11,980 grants, $7,335,036,929 funding
  - 2024: 11,641 grants, $6,375,623,081 funding


In [17]:
# Calculate average metrics for each program element across 2021-2024
for element in tqdm(all_unique_program_elements, desc="Calculating average metrics"):
    # Get proportions and percentages for each year
    yearly_funding_props = [annual_metrics[year][element]['funding_proportion'] for year in baseline_years]
    yearly_grant_pcts = [annual_metrics[year][element]['grant_percentage'] for year in baseline_years]
    
    # Calculate averages
    avg_funding_proportion = sum(yearly_funding_props) / len(baseline_years)
    avg_grant_percentage = sum(yearly_grant_pcts) / len(baseline_years)
    
    average_metrics[element] = {
        'funding_proportion': avg_funding_proportion,
        'grant_percentage': avg_grant_percentage
    }

print("\n2025 metrics:")
print(f"  - Total grants: {annual_totals[2025]['total_grants']:,}")
print(f"  - Total funding: ${annual_totals[2025]['total_funding']:,.0f}")

print("\nAverage calculations completed!")

Calculating average metrics: 100%|██████████| 639/639 [00:00<00:00, 250786.96it/s]


2025 metrics:
  - Total grants: 9,233
  - Total funding: $4,770,031,044

Average calculations completed!


In [18]:
# Display sample average metrics
average_metrics_sample = dict(list(average_metrics.items())[:5])
print("\nSample average metrics (2021-2024):")
for element, metrics in average_metrics_sample.items():
    print(f"  - {element}: {metrics}")


Sample average metrics (2021-2024):
  - Sci of Lrng & Augmented Intel: {'funding_proportion': np.float64(0.0010254687978123785), 'grant_percentage': 0.0011620323235812884}
  - CIS-Civil Infrastructure Syst: {'funding_proportion': np.float64(0.0013516948510208964), 'grant_percentage': 0.0022087249496514716}
  - Cross-Agency Priority (CAP) Go: {'funding_proportion': np.float64(7.689580977709625e-06), 'grant_percentage': 4.1863250462812344e-05}
  - Dimensions of Biodiversity: {'funding_proportion': np.float64(0.000354911846758217), 'grant_percentage': 0.000312130495792353}
  - DAT-Democracy AffrmngTchnolgie: {'funding_proportion': np.float64(0.00016739257919010813), 'grant_percentage': 0.0001669449081803005}


## Step 6: Compile Final Results

In [19]:
# Create final comparison dataframe
print("Compiling final results...")

results_data = []

for element in all_unique_program_elements:
    # Get 2025 metrics
    metrics_2025 = annual_metrics[2025][element]
    metrics_average = average_metrics[element]
    
    # Calculate changes
    funding_prop_change = metrics_2025['funding_proportion'] - metrics_average['funding_proportion']
    funding_relative_change = (funding_prop_change / metrics_average['funding_proportion'] * 100) if metrics_average['funding_proportion'] != 0 else np.nan
    grant_pct_change = metrics_2025['grant_percentage'] - metrics_average['grant_percentage']
    grant_relative_change = (grant_pct_change / metrics_average['grant_percentage'] * 100) if metrics_average['grant_percentage'] != 0 else np.nan
    
    results_data.append({
        'program_element': element,
        'funding_proportion_2025': metrics_2025['funding_proportion'],
        'funding_proportion_2021_2024_avg': metrics_average['funding_proportion'],
        'funding_relative_change': funding_relative_change,
        'funding_proportion_change': funding_prop_change,
        'grant_percentage_2025': metrics_2025['grant_percentage'],
        'grant_percentage_2021_2024_avg': metrics_average['grant_percentage'],
        'grant_percentage_change': grant_pct_change,
        'grant_relative_change': grant_relative_change,
        'grants_2025': metrics_2025['grant_count'],
        'funding_2025': metrics_2025['funding_total']
    })

# Create DataFrame and sort by funding impact
results_df = pd.DataFrame(results_data)

print(f"Results compiled for {len(results_df)} program elements")
print(f"Results dataframe shape: {results_df.shape}")

Compiling final results...
Results compiled for 639 program elements
Results dataframe shape: (639, 11)


In [20]:
# Display summary statistics
print("=== SUMMARY STATISTICS ===")
print(f"\nTotal unique program elements analyzed: {len(results_df):,}")
print(f"\n2025 vs 2021-2024 Average Comparison:")
print(f"  - 2025 total grants: {annual_totals[2025]['total_grants']:,}")
print(f"  - 2025 total funding: ${annual_totals[2025]['total_funding']:,.0f}")
print(f"\n2021-2024 Annual Averages:")
avg_grants = sum(annual_totals[year]['total_grants'] for year in baseline_years) / len(baseline_years)
avg_funding = sum(annual_totals[year]['total_funding'] for year in baseline_years) / len(baseline_years)
print(f"  - Average grants per year: {avg_grants:,.0f}")
print(f"  - Average funding per year: ${avg_funding:,.0f}")

print(f"\nProgram elements with highest funding in 2025:")
top_2025_funding = results_df.nlargest(10, 'funding_proportion_2025')[['program_element', 'funding_proportion_2025', 'grant_percentage_2025']]
print(top_2025_funding.to_string(index=False))

print(f"\nProgram elements with largest funding proportion increases:")
top_increases = results_df.nlargest(10, 'funding_proportion_change')[['program_element', 'funding_proportion_change', 'funding_proportion_2025', 'funding_proportion_2021_2024_avg']]
print(top_increases.to_string(index=False))

print(f"\nProgram elements with largest funding proportion decreases:")
top_decreases = results_df.nsmallest(10, 'funding_proportion_change')[['program_element', 'funding_proportion_change', 'funding_proportion_2025', 'funding_proportion_2021_2024_avg']]
print(top_decreases.to_string(index=False))

=== SUMMARY STATISTICS ===

Total unique program elements analyzed: 639

2025 vs 2021-2024 Average Comparison:
  - 2025 total grants: 9,233
  - 2025 total funding: $4,770,031,044

2021-2024 Annual Averages:
  - Average grants per year: 11,878
  - Average funding per year: $7,275,843,275

Program elements with highest funding in 2025:
               program_element  funding_proportion_2025  grant_percentage_2025
OFFICE OF MULTIDISCIPLINARY AC                 0.036215               0.034117
S-STEM-Schlr Sci Tech Eng&Math                 0.027922               0.007798
                 SBIR Phase II                 0.025738               0.011156
               SHIP OPERATIONS                 0.023922               0.000975
          Cross-BIO Activities                 0.020021               0.014838
             EPSCoR Co-Funding                 0.019052               0.014405
                          IUSE                 0.015573               0.019170
           Software Institutes  

In [21]:
# Filter for program elements with meaningful presence (more than 0.1% funding in both periods)
significant_elements = results_df[
    (results_df['funding_proportion_2025'] >= 0.001) & 
    (results_df['funding_proportion_2021_2024_avg'] >= 0.001)
].copy()

print(f"\n=== SIGNIFICANT PROGRAM ELEMENTS ANALYSIS ===")
print(f"Program elements meeting significance criteria: {len(significant_elements):,}")
print(f"(More than 0.1% funding in both periods)")

print(f"\nTop 20 program elements by 2025 funding proportion:")
top_significant = significant_elements.nlargest(20, 'funding_proportion_2025')
display_cols = ['program_element', 'funding_proportion_2025', 'funding_proportion_2021_2024_avg', 
                'funding_proportion_change', 'grants_2025']
print(top_significant[display_cols].to_string(index=False))


=== SIGNIFICANT PROGRAM ELEMENTS ANALYSIS ===
Program elements meeting significance criteria: 226
(More than 0.1% funding in both periods)

Top 20 program elements by 2025 funding proportion:
               program_element  funding_proportion_2025  funding_proportion_2021_2024_avg  funding_proportion_change  grants_2025
OFFICE OF MULTIDISCIPLINARY AC                 0.036215                          0.045287                  -0.009071          315
S-STEM-Schlr Sci Tech Eng&Math                 0.027922                          0.018952                   0.008969           72
                 SBIR Phase II                 0.025738                          0.018062                   0.007676          103
          Cross-BIO Activities                 0.020021                          0.028494                  -0.008472          137
             EPSCoR Co-Funding                 0.019052                          0.025821                  -0.006769          133
                          I

In [22]:
# Save results to CSV files
print("Saving results to CSV files...")

# Save complete results
results_df.to_csv('export/nsf/nsf_program_element_analysis_complete.csv', index=False)
print(f"  - Complete results saved: nsf_program_element_analysis_complete.csv ({len(results_df)} rows)")

# Save significant program elements only
significant_elements.to_csv('export/nsf/nsf_program_element_analysis_significant.csv', index=False)
print(f"  - Significant elements saved: nsf_program_element_analysis_significant.csv ({len(significant_elements)} rows)")

# Save top changers for easy review
top_changers = pd.concat([
    significant_elements.nlargest(50, 'funding_proportion_change'),
    significant_elements.nsmallest(50, 'funding_proportion_change')
]).drop_duplicates()

top_changers.to_csv('export/nsf/nsf_program_element_analysis_top_changes.csv', index=False)
print(f"  - Top changes saved: nsf_program_element_analysis_top_changes.csv ({len(top_changers)} rows)")

print("\nNSF Program Element Analysis complete! 🎉")

Saving results to CSV files...
  - Complete results saved: nsf_program_element_analysis_complete.csv (639 rows)
  - Significant elements saved: nsf_program_element_analysis_significant.csv (226 rows)
  - Top changes saved: nsf_program_element_analysis_top_changes.csv (100 rows)

NSF Program Element Analysis complete! 🎉
